## Elo Hackathon 2020
### Union Squad –  Daniel Rocha, Letícia Fiorito, Everton Arruda, Juliana Lima
Ideia central: Oferta de cashback por BOT de WhatsApp, SMS ou comando de voz. Ex. “pizza”, e o app mostra opções ao redor (com geocoding) de pizzarias que oferecem cashback para pagamentos com ELO.

Ideia complementar: BOT capta dados de georeferenciamento para dar sugestões e personalizar a oferta de cashback. Dessa forma, é mapeado a territorialidade percorrida pelo consumidor para sugerir cashback em serviços próximos aos locais de trânsito.

Crescimento inicial inorgânico: (isenção de taxas, redução de tarifas) para incentivar aceitação de pagamentos digitais pelo comércio. Após adesão de comerciantes, a contrapartida seria o direcionamento de novos clientes (incentivados pelo cashback) ao estabelecimento, de modo que o restaurante teria que arcar com os custos de manter o cashback (ou optar por cessá-lo).

Cultura cashless: pagamento com QR Code (ex. comanda na pizzaria vem com QR code).

### Importando bibliotecas e fazendo ajustes visuais

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import palettable
import matplotlib.pyplot as plt
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [2]:
for m,n in zip(["display.max_columns","display.max_rows"], [None,10]):
    pd.set_option(m,n)

import matplotlib.style
import matplotlib as mpl
for m,n in zip(['grid.color','axes.titlecolor','axes.labelcolor','xtick.color','ytick.color','axes.titlesize',
    'figure.figsize','legend.framealpha'],['white','white','white','white','white','20.0','13.0, 8.0','0.8']):
    mpl.rcParams[m] = n

### Importando os dados da API

In [29]:
import base64
client_id = 'dfd72f25-8a94-3ba0-8a1b-5a4c0c702718'
client_secret = 'd05a08d7-7ce0-36e0-bc2b-59fd6f4498aa'
base64_str = client_id + ":" + client_secret
base64_bytes = base64_str.encode('utf-8')

authorization = "Basic " + str(base64.b64encode(base64_bytes));
print(authorization)

Basic b'ZGZkNzJmMjUtOGE5NC0zYmEwLThhMWItNWE0YzBjNzAyNzE4OmQwNWEwOGQ3LTdjZTAtMzZlMC1iYzJiLTU5ZmQ2ZjQ0OThhYQ=='


In [46]:
from sgqlc.endpoint.http import HTTPEndpoint

url = 'https://hml-api.elo.com.br/graphql'
headers = {'Authorization': authorization}

query = '''
mutation{
  createUser(input:{
    clientMutationId: "123",
    username: "username",
    bcryptPassword: "$2a$12$oumE.pnRbsdi.c4KYsyNWOJoVyfi2jMKDe102Er5uGCdzd49SvE1Y",
    name: "name",
    firstName: "firstName",
    lastName: "lastName",
        contacts: [{
            type: PHONE
            context: "CASA"
            value: "+5519912345678"
        }]
    displayName: "displayName",
    legalIds:{
      cpf: "1234567890",
      rg:{
        number: "234567890",
        issuerOrganization: "SSP",
        issuerState: "SP",
        issueDate: "2020-12-03"
      }
    }
    origin: "Promoção cadastre sua conta Elo"
  })
  {
    clientMutationId,
    id,
    name,
    oauthToken {
      accessToken
      refreshToken
    }
  }
}
'''

endpoint = HTTPEndpoint(url, headers)
data = endpoint(query)

https://hml-api.elo.com.br/graphql: HTTP Error 401: 


In [53]:
import requests

headers = {"Authorization": authorization}


def run_query(query): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://hml-api.elo.com.br/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}".format(request.status_code))

        
# The GraphQL query (with a few aditional bits included) itself defined as a multi-line string.       
query = '''
mutation{
  createUser(input:{
    clientMutationId: "123",
    username: "username",
    bcryptPassword: "$2a$12$oumE.pnRbsdi.c4KYsyNWOJoVyfi2jMKDe102Er5uGCdzd49SvE1Y",
    name: "name",
    firstName: "firstName",
    lastName: "lastName",
        contacts: [{
            type: PHONE
            context: "CASA"
            value: "+5519912345678"
        }]
    displayName: "displayName",
    legalIds:{
      cpf: "1234567890",
      rg:{
        number: "234567890",
        issuerOrganization: "SSP",
        issuerState: "SP",
        issueDate: "2020-12-03"
      }
    }
    origin: "Promoção cadastre sua conta Elo"
  })
  {
    clientMutationId,
    id,
    name,
    oauthToken {
      accessToken
      refreshToken
    }
  }
}
'''
result = run_query(query) # Execute the query
remaining_rate_limit = result#["data"]["rateLimit"]["remaining"] # Drill down the dictionary
# print("Remaining rate limit - {}".format(remaining_rate_limit))

Exception: Query failed to run by returning code of 401